In [ ]:
import numpy as np
import pandas as pd
import io
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split

In [ ]:
df_true = pd.read_csv('True.csv')

In [ ]:
df_true['label'] = 1

In [ ]:
df_fake = pd.read_csv('Fake.csv')

In [ ]:
df_fake['label'] = 0

In [ ]:
df = pd.concat([df_true,df_fake],ignore_index=True)

In [ ]:
df

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [ ]:
finaldf = df.sample(frac=1).reset_index(drop=True)

In [ ]:
finaldf

,title,text,subject,date,label
0,Researchers raise doubts over cause of Chilean...,SANTIAGO (Reuters) - International researchers...,worldnews,"October 21, 2017",1
1,"Germany softens stance on Turkish arms sales, ...",ANKARA/BERLIN (Reuters) - Chancellor Angela Me...,worldnews,"September 12, 2017",1
2,PARENTS JAILED AND KIDS TAKEN AWAY FOR 90 MINU...,This is so over the top it s not funny. I m a ...,Government News,"Jun 11, 2015",0
3,4th of July: Why the Shadow Government Hates I...,SARTRE 21st Century WireThe only conclusion an...,Middle-east,"July 4, 2017",0
4,Trump administration's Africa policy in focus ...,WASHINGTON (Reuters) - With the Trump administ...,politicsNews,"August 8, 2017",1
...,...,...,...,...,...
44893,RUSH Limbaugh DESTROYS Hillary With Montage Of...,"Not because she s an unlikeable elitist liar, ...",politics,"Dec 11, 2016",0
44894,Pressure rises on Marco Rubio to run for Senat...,WASHINGTON (Reuters) - U.S. Senate Majority Le...,politicsNews,"May 31, 2016",1
44895,Trump's jobs homecoming a long shot even in ma...,"JACKSONVILLE, Fla (Reuters) - When U.S. manufa...",politicsNews,"April 14, 2016",1
44896,ABSOLUTE SUBMISSION: Trump Bows to Neocon Orth...,Consortium News Exclusive: In his Mideast trip...,Middle-east,"June 1, 2017",0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(finaldf.title, finaldf.label)

In [ ]:
X_train

16178    NEWT GINGRICH: Boost the Economy with This or ...
20440    BUMBLING BILL CLINTON Manages To Insult Poland...
36867    No angst over Turkey's air defense deal with R...
14296    Trump aide named to lead White House communica...
31343    Turkey's Justice Ministry says it canceled del...
                               ...                        
35200    Factbox: The future of FBI's Russia probe afte...
24344    Clinton offers plan to freeze some college deb...
12838    Germany will strive to save Iran nuclear deal:...
32165    Japan court sentences U.S. base worker to life...
31387           Northern Ireland: Life inside The Fountain
Name: title, Length: 33673, dtype: object

In [ ]:
pd.set_option('display.max_colwidth', None)
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 64
N_EPOCHS = 5

In [ ]:
#define the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
#tokenize the text
train_encodings = tokenizer(list(X_train.values),
                            truncation=True, 
                            padding=True)
test_encodings = tokenizer(list(X_test.values),
                           truncation=True, 
                           padding=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

In [ ]:
print(f'First paragraph:\'{X_train[:1]}\'')
print(f'Attention mask:{train_encodings["attention_mask"][0]}')
print(f'Input ids:{train_encodings["input_ids"][0]}')

First paragraph:'16178    NEWT GINGRICH: Boost the Economy with This or There’s a ‘Real Danger of Speaker Pelosi’ [Video]
Name: title, dtype: object'
Attention mask:[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Input ids:[101, 25597, 18353, 16523, 7033, 1024, 12992, 1996, 4610, 2007, 2023, 2030, 2045, 1521, 1055, 1037, 1520, 2613, 5473, 1997, 5882, 21877, 10483, 2072, 1521, 1031, 2678, 1033, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),list(y_train.values)))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),list(y_test.values)))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
#pick the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define loss function 
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
#fit the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Epoch 1/5
527/527 [==============================] - 383s 701ms/step - loss: 0.0968 - accuracy: 0.9639
Epoch 2/5
527/527 [==============================] - 370s 701ms/step - loss: 0.0243 - accuracy: 0.9920
Epoch 3/5
527/527 [==============================] - 369s 701ms/step - loss: 0.0114 - accuracy: 0.9960
Epoch 4/5
527/527 [==============================] - 369s 701ms/step - loss: 0.0045 - accuracy: 0.9983
Epoch 5/5
527/527 [==============================] - 369s 701ms/step - loss: 0.0060 - accuracy: 0.9979


In [ ]:
#evaluation on the test set
model.evaluate(test_dataset.shuffle(len(X_test)).batch(BATCH_SIZE), 
               return_dict=True, 
               batch_size=BATCH_SIZE)

176/176 [==============================] - 43s 236ms/step - loss: 0.0525 - accuracy: 0.9877


{'accuracy': 0.9877060055732727, 'loss': 0.05253768712282181}

**Record: **

Setting 1. Samplesize = 0.01, Epoch = 3, Batchsize = 16, 
8/8 [==============================] - 13s 1s/step - loss: 0.3584 - accuracy: 0.8761
{'accuracy': 0.8761062026023865, 'loss': 0.35844138264656067 


Setting 2. Samplesize = 0.01, Epoch = 3, Batchsize = 64, 2/2 [==============================] - 10s 4s/step - loss: 0.2966 - accuracy: 0.8761
{'accuracy': 0.8761062026023865, 'loss': 0.2966170608997345} 

Setting 3. Samplesize = 0.01, Epoch = 5, Batchsize = 64, 2/2 [==============================] - 10s 4s/step - loss: 0.3095 - accuracy: 0.8938
{'accuracy': 0.8938053250312805, 'loss': 0.30945825576782227}

Setting 4(GPU). Samplesize = 1, Epoch = 5, Batchsize = 64, 176/176 [==============================] - 43s 236ms/step - loss: 0.0525 - accuracy: 0.9877
{'accuracy': 0.9877060055732727, 'loss': 0.05253768712282181}
